# Выбор данных

Создадим корпус из переписок чата дубков. Это может быть полезно для поиска информации, связанной с общежитием: когда работает кастелянная, как забронировать досуговую и т.п.

In [1]:
import heapq
import os
import json
import pandas as pd
from typing import List, Union, Tuple
import inspect
import wget
import zipfile
import gensim
from tqdm import tqdm
import numpy as np
import pickle
from nltk.corpus import stopwords
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)
from gensim.models.word2vec import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer


C:\Users\79998\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
with open("./infopoisk HW1/dubki_res.json", 'r', encoding = 'utf-8') as file:
    data_dubki = json.load(file)

In [3]:
len(data_dubki['messages'])

646346

Возьмем часть записей

In [4]:
messages = data_dubki['messages'][:5000]

In [5]:
messages[0]

{'id': 1,
 'type': 'service',
 'date': '2020-09-07T23:31:35',
 'date_unixtime': '1599510695',
 'actor': 'Студенческий городок Дубки',
 'actor_id': 'channel1278030013',
 'action': 'migrate_from_group',
 'title': 'Дубки',
 'text': '',
 'text_entities': []}

Возьмем только те, где есть текст

In [6]:
text_messages = []
for m in messages:
    if m['text']:
        text = m['text']
        if not isinstance(text, list): #берем только без вложений
            text_messages.append(text)

In [7]:
len(text_messages)

4148

Создадим таблицу, в которой будем хранить исходный текст, обработанный текст и два индекса

In [8]:
data = pd.DataFrame(columns=['texts', 'lemmas_lists', 'lemmas_texts', 'index_w2v', 'index_tfidf'])

In [9]:
data['texts'] = text_messages

In [10]:
data.head()

,texts,lemmas_lists,lemmas_texts,index_w2v,index_tfidf
0,Я верю в развитие этого чата,NaN,NaN,NaN,NaN
1,Привет,NaN,NaN,NaN,NaN
2,Есть тусяо?,NaN,NaN,NaN,NaN
3,"Народ, у кого-нибудь есть колонка проводная? З...",NaN,NaN,NaN,NaN
4,Шалом православным,NaN,NaN,NaN,NaN


# Предобработка

Далее 
* лемматизируем 
* уберем стоп-слова
* уберем пунктуацию

Это необходимо для дальнейшей векторизации так как
1. ускоряет работу tf-idf (не надо делать то же во время подсчета)
2. делает обработку с помощью word2vec осмысленной, так как мы выучиваем эмбеддинги только отедльных и значимых слов 

Для лемматизации русского текста лучше всего подойдет natsha - хорошо работает для русского и занимает не много места, что будет важно при создании веб-приложения

In [11]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

sw = stopwords.words('russian')

Хранить предобработанные тексты нужно в двух видах: лист лемм с частью речи для W2V и леммы через пробел для tf-idf

In [12]:
def preprocess(text: str) -> List[Tuple[str, str]]:
    """
    Предобрабатывает входной текст, выполняя токенизацию, приведение к нижнему регистру
    и фильтрацию стоп-слов и неалфавитных и нечисловых токенов.
   
    Args:
        text (str):  Входной текст для предобработки.
    
    Returns:
        List[Tuple[str, str]]:  Лист, содержащий леммы и части их речи. (lemmas).
    """
    lemmas = []
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        token_str = str(token.lemma).lower()
        if token_str.isalpha() or token_str.isnumeric():  # числовые токены включаются
            if token_str not in sw:
                lemmas.append((token_str, token.pos))
    return lemmas

In [13]:
preprocess(text_messages[20])

[('это', 'PRON'),
 ('мистический', 'ADJ'),
 ('выброс', 'NOUN'),
 ('весь', 'DET'),
 ('след', 'NOUN'),
 ('неделя', 'NOUN'),
 ('собираться', 'VERB'),
 ('против', 'ADP'),
 ('надоесть', 'VERB'),
 ('нло', 'NOUN')]

In [14]:
def preprocess_pos_and_text(preprocessed_lemmas: List[Tuple[str, str]]) -> Union[Tuple[List[List[str]], List[str]], None]:
    """
    Преобразует лемматизированные и размеченные по чати речи тексты в кортеж из двух спиков, содержащих данные для удобной векторизации
   
    Args:
       preprocessed_lemmas (List[Tuple[str, str]]):  Предобработанные тексты в виде лемм с частями речи.
    
    Returns:
        List[List[str]]:  Кортеж листов, удобных для работы с Word2vec и TF-IDF.
    """

    lemmas_list_pos = [str(lemma[0]) + '_' + str(lemma[1]) for lemma in preprocessed_lemmas]
    lemmas_text = ' '.join([str(lemma[0]) for lemma in preprocessed_lemmas])
            
    return (lemmas_list_pos, lemmas_text)

In [15]:
lemma_lists_pos = []
lemma_texts = []

for text in tqdm(text_messages):
    preprocessed_lemmas = preprocess_pos_and_text(preprocess(text))
    if preprocessed_lemmas:
        lemma_lists_pos.append(preprocessed_lemmas[0])
        lemma_texts.append(preprocessed_lemmas[1])
    else:
        lemma_lists_pos.append(None)
        lemma_texts.append(None)

100%|██████████| 4148/4148 [00:09<00:00, 457.71it/s]


In [16]:
lemma_lists_pos[20]

['это_PRON',
 'мистический_ADJ',
 'выброс_NOUN',
 'весь_DET',
 'след_NOUN',
 'неделя_NOUN',
 'собираться_VERB',
 'против_ADP',
 'надоесть_VERB',
 'нло_NOUN']

In [17]:
lemma_texts[20]

'это мистический выброс весь след неделя собираться против надоесть нло'

In [18]:
data['lemmas_texts'] = lemma_texts
data['lemmas_lists'] = lemma_lists_pos

In [19]:
data.dropna(subset=['lemmas_texts', 'lemmas_lists'], inplace=True)

In [20]:
data.shape

(4148, 5)

# Word2Vec

In [21]:
model_url = 'http://vectors.nlpl.eu/repository/20/180.zip' # нкря

In [22]:
%%time
m = wget.download(model_url)
model_file = model_url.split('/')[-1]

CPU times: total: 3.5 s
Wall time: 2min 11s


In [23]:
%%time
with zipfile.ZipFile(model_file, 'r') as archive:
    stream = archive.open('model.bin')
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

CPU times: total: 2.89 s
Wall time: 2.94 s


In [24]:
def document_vector(doc: List[str], model: Word2Vec) -> Union[np.ndarray, np.array]:
    """
    Преобразует документ в вектор с помощью усреднения векторов слов из модели Word2Vec.

    Args:
        doc (List[str]): Список токенов (слов) документа.
        model (Word2Vec): Предварительно обученная модель Word2Vec, из которой извлекаются векторы слов.

    Returns:
        Union[np.ndarray, np.array]: Усреднённый вектор документа. Если все слова отсутствуют в модели, возвращается нулевой вектор.
    """
    word_vectors = [model[word] for word in doc if word in model]
    
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(model.vector_size)

In [25]:
w2v_doc_vec = [list(document_vector(text, model)) for text in data['lemmas_lists']]

In [26]:
data['index_w2v'] = w2v_doc_vec

В дальнейшем можно так же обрабатывать и получать из модели вектор, и сравнивать его с помощью косинусной близости

# tf-idf

In [27]:
vectorizer = TfidfVectorizer()

In [28]:
tfidf = vectorizer.fit_transform(data['lemmas_texts'])

In [29]:
tfidf.shape

(4148, 5284)

In [30]:
npm_tfidf = tfidf.toarray()

In [31]:
tfidf_vecs = [list(vec) for vec in npm_tfidf]

In [32]:
data['index_tfidf'] = tfidf_vecs

In [33]:
data.head()

,texts,lemmas_lists,lemmas_texts,index_w2v,index_tfidf
0,Я верю в развитие этого чата,"[верить_VERB, развитие_NOUN, чат_NOUN]",верить развитие чат,"[0.10236853, -0.441486, 2.36323, -0.5347944, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,Привет,[привет_X],привет,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,Есть тусяо?,[тусяо_NOUN],тусяо,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"Народ, у кого-нибудь есть колонка проводная? З...","[народ_NOUN, колонка_NOUN, проводная_NOUN, шок...",народ колонка проводная шоколадка,"[1.7437725, 0.825912, 0.62153894, -0.6994569, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,Шалом православным,"[шалый_PROPN, православный_ADJ]",шалый православный,"[-0.8455712, 0.7992621, 0.2846047, -1.3858058,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


# Облегчение поиска: нормализация векторов заранее
Запишем норму векторов, чтобы при подсчете косинусной близости надо было делать меньше операций

In [34]:
def normalize(vec):
    return np.linalg.norm(vec)

In [35]:
%%time
data['w2v_norm'] = [normalize(vec) for vec in data['index_w2v']]
data['tfidf_norm'] = [normalize(vec) for vec in data['index_tfidf']]

CPU times: total: 4.25 s
Wall time: 1.97 s


In [36]:
# Проверка количества документов с нулевой нормой
zero_norm_w2v = data[data['w2v_norm'] == 0].shape[0]
print(f'Количество документов с нулевой нормой Word2Vec: {zero_norm_w2v}')

zero_norm_tfidf = data[data['tfidf_norm'] == 0].shape[0]
print(f'Количество документов с нулевой нормой TF-IDF: {zero_norm_tfidf}')

Количество документов с нулевой нормой Word2Vec: 702
Количество документов с нулевой нормой TF-IDF: 312


In [37]:
zero_norm_docs_w2v = data[data['w2v_norm'] == 0]['texts'].head()
print('Документы с нулевой нормой Word2Vec:')
print(zero_norm_docs_w2v)

Документы с нулевой нормой Word2Vec:
1          Привет
2     Есть тусяо?
13              ?
52          Движ?
53          Гдеее
Name: texts, dtype: object


In [38]:
zero_norm_docs_tfidf = data[data['tfidf_norm'] == 0]['texts'].head()
print('Документы с нулевой нормой tfidf:')
print(zero_norm_docs_tfidf)

Документы с нулевой нормой tfidf:
13        ?
93       да
100      ❤️
123    Есть
125     ???
Name: texts, dtype: object


Из-за того, что некоторые сообщения состоят из одного слова, которые W2V не знает, или просто из символов, и вектор, и норма равны нулю. Чтобы избежать деления на 0 при подсчете косинусной близости, нам приедтся выкинуть эти записи. Их довольно много, но они все равно не несут много смысла

In [39]:
data = data[data['w2v_norm'] != 0].reset_index(drop=True)
data = data[data['tfidf_norm'] != 0].reset_index(drop=True)

In [40]:
data.shape

(3446, 7)

#  Поиск

реализуем функции поиска

In [41]:
def cosine_sim(vec1, vec2, norm1, norm2):
    scalar_mul = np.dot(vec1, vec2)
    return scalar_mul / (norm1 * norm2)

In [42]:
def search(query: str, search_type: str, top_n: int) -> pd.Series:
    """
    Выполняет поиск наиболее похожих текстов в базе данных на основе заданного запроса и метода поиска.

    Args:
        query (str): Входной текст запроса, который нужно искать.
        search_type (str): Тип поиска. Возможные значения:
            - 'w2v' для поиска на основе word2vec векторов.
            - 'tfidf' для поиска на основе TF-IDF.
        top_n (int): Количество наиболее похожих текстов, которые следует вернуть.

    Returns:
        pd.Series: Серия из топ-N текстов, наиболее похожих на запрос.

    Этапы:
    1. Предобрабатывает запрос с использованием функции `preprocess_pos_and_text`.
    2. В зависимости от выбранного типа поиска ('w2v' или 'tfidf') извлекает вектор запроса.
    3. Нормализует вектор запроса.
    4. Вычисляет косинусное сходство между вектором запроса и векторами в базе данных.
    5. Возвращает топ-N текстов с наибольшим сходством.
    """
    preprocessed_q = preprocess_pos_and_text(preprocess(query))
    
    if search_type == 'w2v':
        preprocessed_q = preprocessed_q[0]
        q_vector = document_vector(preprocessed_q, model)
        index_name = 'index_w2v'
        norm_name = 'w2v_norm'
        
    elif search_type == 'tfidf':
        preprocessed_q = preprocessed_q[1]
        q_vector = vectorizer.transform([preprocessed_q]).toarray()[0]
        index_name = 'index_tfidf'
        norm_name = 'tfidf_norm'
        
    q_norm = normalize(q_vector)
    
    similarities = []
    for idx, vec, norm in zip(data.index, data[index_name], data[norm_name]):
        sim = cosine_sim(q_vector, vec, q_norm, norm)
        similarities.append((sim, idx))
        
    top_similarities = heapq.nlargest(top_n, similarities, key=lambda x: x[0])
    top_idxs = [idx for _, idx in top_similarities]    
    top_texts = data.loc[top_idxs, 'texts']
    return top_texts


In [43]:
query = 'Где достать еду в дубках'

In [44]:
%%time
res = search(query, 'w2v', 5)
for i in res:
    print(i)
    print('_____')

Ладно хоть еду приносят
_____
Через вк еда много доставок есть
_____
Съем свои купленные чипсы.
_____
Я за супом то не пошел, а тут ложка сметаны
_____
ну засунь в морозилку
_____
CPU times: total: 594 ms
Wall time: 168 ms


In [45]:
%%time
res = search(query, 'tfidf', 5)
for i in res:
    print(i)
    print('_____')

Еду с КГ
_____
Через вк еда много доставок есть
_____
Андрей достал - прими гастал
_____
Куда пойти в дубках?
_____
Ладно хоть еду приносят
_____
CPU times: total: 4.52 s
Wall time: 1.55 s


Итог: поиск работает очень быстро и достаточно информативно

# Сохраняем для дальнейшего использования
* функции предобработки
* функции нормализации и подсчета косинусной близости
* модель w2v и функцию векторизации
* векторайзер tf-idf
* полученную табличку

In [46]:
# сохраняем функции предобработки
preprocess_code = inspect.getsource(preprocess)
preprocess_pos_and_text_code = inspect.getsource(preprocess_pos_and_text)

with open("./infopoisk HW1/preprocess_functions.py", 'w', encoding = 'utf-8') as file:
    file.write(preprocess_code +'\n\n')    
    file.write(preprocess_pos_and_text_code +'\n')

In [47]:
# сохраняем функции нормы и косинусной близости
normalize_code = inspect.getsource(normalize)
cosine_sim_code = inspect.getsource(cosine_sim)

with open("./infopoisk HW1/math_functions.py", 'w', encoding = 'utf-8') as file:
    file.write(normalize_code +'\n\n')    
    file.write(cosine_sim_code +'\n')

In [48]:
# сохраняем функцию векторизации текста с помощью W2V
docvec_code = inspect.getsource(document_vector)

with open("./infopoisk HW1/docvec_function.py", 'w', encoding = 'utf-8') as file:
    file.write(docvec_code +'\n')

In [49]:
# сохраняем функцию поиска
search_code = inspect.getsource(search)

with open("./infopoisk HW1/search_function.py", 'w', encoding = 'utf-8') as file:
    file.write(search_code +'\n')

In [50]:
# сохраняем векторайзер
with open('./infopoisk HW1/tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

In [51]:
#сохраняем модель w2v
model.save("./infopoisk HW1/w2v_model.bin")  # Сохранение в KeyedVectors формате

In [52]:
%%time
#сохраняем все данные
data.to_csv('./infopoisk HW1/data.csv')

CPU times: total: 4.56 s
Wall time: 4.92 s


Теперь используем это для реализации сайта с поисковиком

# Формальное сравнение двух индексаций

Используя весь уже имеющийся инструментарий, можем сравнить, насколько хорошо модели/векторайзеры справляются с поиском как бы по собственному мнению, а именно, посмотреть на косинусные близости. Чем больше косинусная близость между запросом и top_n, тем круче работает поиск.

In [53]:
def get_similarities(query: str, top_n: int) -> pd.Series:
    
    preprocessed_q = preprocess_pos_and_text(preprocess(query))
    
    preprocessed_q_w2v = preprocessed_q[0]
    q_vector_w2v = document_vector(preprocessed_q_w2v, model)
    index_name_w2v = 'index_w2v'
    norm_name_w2v = 'w2v_norm'
        
    preprocessed_q_tfidf = preprocessed_q[1]
    q_vector_tfidf = vectorizer.transform([preprocessed_q_tfidf]).toarray()[0]
    index_name_tfidf = 'index_tfidf'
    norm_name_tfidf = 'tfidf_norm'
    
    q_norm_w2v = normalize(q_vector_w2v)   
    q_norm_tfidf = normalize(q_vector_tfidf)
        
    similarities_w2v = []
    similarities_tfidf = []
    
    # считаем сходства для w2v
    for vec, norm in zip(data[index_name_w2v], data[norm_name_w2v]):
        sim = cosine_sim(q_vector_w2v, vec, q_norm_w2v, norm)
        similarities_w2v.append(sim)
        
    # считаем сходства для tf-idf    
    for vec, norm in zip(data[index_name_tfidf], data[norm_name_tfidf]):
        sim = cosine_sim(q_vector_tfidf, vec, q_norm_tfidf, norm)
        similarities_tfidf.append(sim)
        
    top_similarities_w2v = heapq.nlargest(top_n, similarities_w2v, key=lambda x: x)
    top_similarities_tfidf = heapq.nlargest(top_n, similarities_tfidf, key=lambda x: x)

    return top_similarities_w2v, top_similarities_tfidf


In [54]:
queries = ['Где найти еду',
           'Почему дежурка приходит так часто',
           'Как отсюда уехать',
           'тусовки со студсоветом',
          'почему небо голубое',
          'расписание сессий НИУ ВШЭ']

In [55]:
for q in queries: 
    print(q)
    sims = get_similarities(q, 5)
    for wv, tf in zip(sims[0], sims[1]):
        print(round(wv, 3), '\t', round(tf, 3))
    print('- - - - - - - - - - - - - - - -')

Где найти еду
0.676 	 0.596
0.609 	 0.596
0.585 	 0.596
0.585 	 0.542
0.585 	 0.492
- - - - - - - - - - - - - - - -
Почему дежурка приходит так часто
0.572 	 0.578
0.572 	 0.452
0.572 	 0.399
0.572 	 0.398
0.572 	 0.398
- - - - - - - - - - - - - - - -
Как отсюда уехать
0.727 	 0.436
0.589 	 0.409
0.578 	 0.359
0.511 	 0.309
0.473 	 0.306
- - - - - - - - - - - - - - - -
тусовки со студсоветом
1.0 	 0.732
1.0 	 0.732
0.614 	 0.471
0.61 	 0.387
0.581 	 0.344
- - - - - - - - - - - - - - - -
почему небо голубое
0.461 	 0.627
0.422 	 0.54
0.422 	 0.54
0.422 	 0.54
0.422 	 0.54
- - - - - - - - - - - - - - - -
расписание сессий НИУ ВШЭ
0.62 	 0.557
0.617 	 0.321
0.613 	 0.308
0.613 	 0.295
0.581 	 0.295
- - - - - - - - - - - - - - - -


На глаз кажется, что W2V в среднем справляется лучше

Далее как cамый просто формального сравнения - вычесть попарно сходства и найти среднее. Получим условную величину того, насколько один индекс лучше другого

In [56]:
top_n = 15
for q in queries: 
    print(q)
    sims = get_similarities(q, top_n)
    subs = []
    for wv, tf in zip(sims[0], sims[1]):
        subs.append(wv - tf)
    print(sum(subs)/top_n)
    print('- - - - - - - - - - - - - - - - - - -')

Где найти еду
0.077420120813861
- - - - - - - - - - - - - - - - - - -
Почему дежурка приходит так часто
0.1451854333103196
- - - - - - - - - - - - - - - - - - -
Как отсюда уехать
0.2772900666108932
- - - - - - - - - - - - - - - - - - -
тусовки со студсоветом
0.28725473337038815
- - - - - - - - - - - - - - - - - - -
почему небо голубое
-0.03953198423675864
- - - - - - - - - - - - - - - - - - -
расписание сессий НИУ ВШЭ
0.2560921190297827
- - - - - - - - - - - - - - - - - - -


Большинство ответов положительные (хоть иногда и близкие к нулю), следовательно победу можно присудить Word2Vec. В дальнейшем для более точного анализа можно посмотреть на среднне аналогичной метрики по большому количеству запросов (например, сгенерированных нейросетью), делить эти запросы на категории (например, запросы связанные с ВШЭ, абстрактные запросы и т.п.), брать больше реpультатов и учитывать при вычислениях их порядок выдачи